<a id = 'top'></a>

#  A quick-start guide to fine-tune a BERT model with Keras
  * A. [What is BERT?](#introBERT)
  * B. [What is fine-tuning?](#fineTuned)
  * C. [Datasets](#datasetClass)
      * 1. [IMDB Description](#IMDBdesc)
      * 2. [Exploratory Data Analysis](#EDA)
  * D. [Model Preparation](#modelPrep)
      * 1. [Model Selection](#modelSelection)
      * 2. [Tokenizer Selection](#tokenizerSelect)
      * 3. [Auto Model](#autoModel)
      * 4. [Encode Data](#encodeData)
  * E. [Fine-Tuning The Model](#fineTuning)
     

Hugging Face is a company that offers a library of "transformers" as well as a collection of pre-trained language models.  These represent one source of code and abstract classes as well as a variety of documentation and examples. We are going to explore one way of working with these models at a very high level.  In later classes, when we have covered how a transformer works, we'll come back and look at them at a deeper level.  This tutorial is designed to look at the Huggingface library at the same level of abstraction as the Keras Sequential API rather at the lower level of abstraction of TensorFlow and the Keras Functional API.

---

Larger models, with millions or billions of paramters can only be trained on a machine with a GPU.  Do not run this notebook on your GCP instance as the training epochs will take entirely too long.  If you run this notebook in a Colab, it is automatically configured to use a GPU.


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/datasci-w266/2023-fall-main/blob/master/materials/walkthrough_notebooks/bert_as_black_box/Keras_HuggingFace_Transformers_BERT_notebook.ipynb)

[Return to Top](#top)
 <a id = 'introBERT'></a>
# What is BERT?
This notebook leverages one of a variety of BERT models.  BERT models can be classified in terms of three parts.  The first part is a component named a transformer.  These can grow to be quite large.  BERT consists of either 12 or 24 layers of transformers. The second part is the training (called pre-training) the model already has on language.  The pre-training is characterized by one or more tasks.  The third part consists of the very specific tasks it is geared toward performing.  Different models use different sizes and layers of transformers and may be optimized for different languages and different tasks.  For example, CamemBERT is trained in French and SciBERT is trained on scientific journal articles.  You'll want to make sure you use a model appropriate to your language and task.

---

The [HuggingFace web site](https://huggingface.co/transformers) offers an interesting set of resources.  Their [model documentation](https://huggingface.co/transformers/model_summary.html) provides an excellent explanation of transformers as well as the growing variety of models they offer (see the right hand navigation column).  In addition, their collection of [notebooks](https://huggingface.co/transformers/notebooks.html) is a valuable set of examples.  

---

One word of caution:  this is a rapidly evolving resource and as a result you can often run in to bugs.  They will get fixed, eventually, but may be buggy for a while.  

[Return to Top](#top)
 <a id = 'fineTuned'></a>
 # Fine-tuning a Model

We'll use abstract classes that simplify the process of training by consolidating a number of pieces under one class. It's a good way to begin working with these models.  PyTorch is the native computational graph language used in Hugging Face. However, they make a point of porting models to TensorFlow, Google's computational graph language. Many models first get put on HuggingFace in PyTorch. Eventually they get ported over to TensorFlow. Depending on what model you want to use, you may have to run the PyTorch version. It's important to always be aware of which dialect you're using. The good news is that HuggingFace has built these models so that the underlying weight parameters can be used across PyTorch and TensorFlow implementations. It is simply the commands you use to construct, run, and manipulate the model that are in PyTorch or TensorFlow.  This notebook will demonstrate fine-tuning a TensorFlow HuggingFace model using Keras.  To do this, we'll need to select a data set, a model, and be sure to invoke its tokenizer.

 Borrowing liberally from the fine-tuning description in https://huggingface.co/transformers/training.html

In [ ]:
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 85.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.3 MB/s eta 0:00:00


[Return to Top](#top)
 <a id = 'datasetClass'></a>
# Datasets



HuggingFace provides [a class for the managing datasets](https://huggingface.co/docs/datasets/index). They also provide a library of actual data that is accessible via this datasets class. We'll take advantage of the datasets object in Huggingface to access some well known corpora, specifically IMDB.

In [ ]:
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00


[Return to Top](#top)
 <a id = 'IMDBdesc'></a>
### IMDB Description

IMDB is a set of movie reviews. It is set up for a binary sentiment classification task.  It is good for learning how to work with HuggingFace Transformers library and also good for baselines.

In [ ]:
from datasets import load_dataset

raw_datasets = load_dataset("imdb")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

[Return to Top](#top)
 <a id = 'EDA'></a>
### Exploratory Data Analysis

Let's look inside the IMDB dataset and see what it contains.  We see it is already split into train, test, and unsupervised records.  

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

Here is one sample record from the test set.  Each record contains a label and some text.  Different data sets will have different parts in their records.

In [ ]:
raw_datasets['test'][0]

{'text': 'I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a \'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it\'s rubbish as 

Here is a utility function that leverages the dataset structure to display 10 random records from the dataset and loads them in a data frame.

In [ ]:
#from https://github.com/huggingface/notebooks/blob/master/examples/text_classification.ipynb
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(raw_datasets['test'])

,text,label
0,"I think Josh Duhamel is so great!! The rest of the show is fun to watch, but, I think it is the handsome and sexy Josh Duhamel that makes the show ""Las Vegas"" really fun to see!! In the days of ""Magnum"" I loved Tom Selleck, I thought he was the sexiest man on the face of the earth!! A hunk on a television show is a must in order for women to enjoy watching something, especially just for purposes of innocuous entertainment!! I would have done anything to ""Win A Date With Tad Hamilton""!! Josh Duhamel is incredible and I will always have a super crush on him!! Josh is definitely a HUNK!! and I will watch ""Las Vegas"" all the time, Josh Duhamel is a big reason why too!!",pos
1,"This film powerfully demonstrates the struggle of two women in love in a culture so deeply entrenched in ritual and tradition. All this against a backdrop of an India which itself is struggling for freedom from these same values. This film is both political and personal and never too preachy or idealistic on either front. It is easy to see why ""Fire"" has caused riots in India, but tragic nonetheless. A true film such as this one deserves to be seen by all people of the world, not just privileged westerners.",pos
2,"For some unknown reason, 7 years ago, I watched this movie with my mother and sister. I don't think I've ever laughed as hard with them before. This movie was sooooo bad. How sequels were produced is beyond me. Its been awhile since I last saw this ""movie"", but the one impression that it has stuck with me over the years has been, ""They must have found the script in a dumpster in the backlot of a cheap movie studio, made into a ""movie"", and decided that it didn't suck enough, and made it worse. I'm pretty sure that they spent all the budget on camera work and the so called ""special effects"", and then had 13 cents left toward the script AND to pay the ""actors"".",neg
3,"Spending an hour seeing this brilliant Dan Finnerty and his ""Dan Band"" perform their special on Bravo is the most enjoyable hour I've ever spent watching TV. This young man (Dan) is such an incredible talent, as a singer, performer and even dancer. He can go from the cheesiest of ballad pop songs, all of which have only been sung by women, to hip-hop, rock, also songs written for women.. This guy can do anything. I've seen him live at least 11 times, so I was not expecting just how well that his show would adapt to a television or film format, but all reservations went away instantly when the show started because of Dan's overwhelming star quality.Do yourself a favor and watch this, or better yet, buy it.",pos
4,"The true story of a Spanish paraplegic, Ramón Sampedro, who fought for decades for the right to be euthenized. This film, along with the Best Picture winner of the same year, Million Dollar Baby, caused a stir that year with their depictions of disabled persons desiring death. Both advocates for the disabled and (unfortunately for the disability advocates) conservative pro-life groups protested both films, and their Oscar nominations. The nominations also came during the entire Terry Schiavo debacle, just to put it all in some historical perspective. The protests, especially from the disability groups, against Million Dollar Baby make some sense  the film clearly depicted, without wavering, the life of a paraplegic as worthless. The film's central character, Maggie Fitzgerald, becomes a paraplegic, doesn't seem to get any counseling whatsoever, no help whatsoever, and immediately wants to die. The film is, honestly, pretty dumb and uncomplex. The Sea Inside, based on the true story, is certainly a lot more thoughtful on the subject. It most likely got railroaded into the same category as Million Dollar Baby without its protesters having even seen it, an incredibly common phenomenon. The film does give time to many different sides of the argument. And it immediately declares that the wish to die is that of the protagonist and the protagonis

[Return to Top](#top)
 <a id = 'modelPrep'></a>
## Model Preparation

[Return to Top](#top)
 <a id = 'modelSelection'></a>
### Model Selection

We need to pick the model we are going to train to classify IMDB.  We'll do that in several stages. First we define some variables to hold information about the model that we'll re-use.

In [ ]:
model_checkpoint = "bert-base-cased"
batch_size = 8

[Return to Top](#top)
 <a id = 'tokenizerSelect'></a>
### Tokenizer Selection

We'll use the [AutoTokenizer](https://huggingface.co/docs/transformers/main/en/autoclass_tutorial#autotokenizer) object to avoid simple configuration mistakes because it insures that we get the correct tokenizer given our pre-trained model.  This time the model we're using is BERT and we're selecting the cased version (meaning case is preserved) and the base version (rather than the large version).

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

What's the tokenizer doing?  It's taking care of breaking down a sentence into the parts the model can understand and was trained on, as well as a bunch of housekeeping that's needed by the model in order to work properly.  Once we've covered how a transformer works in live session, we'll come back (in week 4) and discuss its various components.  For now, you don't need to understand it in order to make use of it.

Here's one example of what the tokenizer outputs.

In [ ]:
tokenizer("Hello, we only need one sentence for our task but these reviews often have more.")

{'input_ids': [101, 8667, 117, 1195, 1178, 1444, 1141, 5650, 1111, 1412, 4579, 1133, 1292, 3761, 1510, 1138, 1167, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

The tokenizer converts the incoming words to integer ids that are used to retrieve the model's input word embeddings.  All tokenizers convert words to input ids.  The wrong tokenizer will produce the wrong set of token ids and result in very poor predictions.  The AutoTokenizer insures the correct ids are assigned.

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))
full_train_dataset = tokenized_datasets["train"]
full_eval_dataset = tokenized_datasets["test"]

[Return to Top](#top)
 <a id = 'autoModel'></a>
### Automatic Model Configuration

We'll use the [AutoModel abstraction](https://huggingface.co/docs/transformers/main/en/autoclass_tutorial#automodel) and invoke the TensorFlow port since we want to use Keras to train and run the model.  As a result we'll instantiate a copy of TFAutoModelForSequenceClassification.  Note the 'TF' at the begining of the class name to designate it as a TensorFlow port.  The model for "sequence classification" is specifically structured to perform classification based on sequences of words like a sentence.  HuggingFace provides a set of models specifically configured for [particular NLP tasks](https://huggingface.co/docs/transformers/main/en/model_doc/auto) as shown by all of the AutoModelFor *FillInTheTask*.

In [ ]:
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[Return to Top](#top)
 <a id = 'encodeData'></a>
### Encode Data


Let's create the encoded data for training.  Because we're using the TensorFlow port, we'll need to convert our PyTorch dataset object contents to the TensorFlow version.  HuggingFace provides some nice conversion functions to assist in the process.

In [ ]:
tf_train_dataset = small_train_dataset.remove_columns(["text"]).with_format("tensorflow")
tf_eval_dataset = small_eval_dataset.remove_columns(["text"]).with_format("tensorflow")

In [ ]:
train_features = {x: tf_train_dataset[x] for x in tokenizer.model_input_names}
train_tf_dataset = tf.data.Dataset.from_tensor_slices((train_features, tf_train_dataset["label"]))
train_tf_dataset = train_tf_dataset.shuffle(len(tf_train_dataset)).batch(batch_size)

eval_features = {x: tf_eval_dataset[x] for x in tokenizer.model_input_names}
eval_tf_dataset = tf.data.Dataset.from_tensor_slices((eval_features, tf_eval_dataset["label"]))
eval_tf_dataset = eval_tf_dataset.batch(batch_size)

[Return to Top](#top)
 <a id = 'fineTuning'></a>
# Fine Tuning

In keeping with the Keras process we call model.compile first to make sure that all the pieces are in place.  We can follow that up with a call to model.summary() to make sure we've put the correct players together in the correct manner.  We can also see how much is trainable, which gives a sense of training time and resource requirements.

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=tf.metrics.SparseCategoricalAccuracy(),
)

In [ ]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  108310272 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 108311810 (413.18 MB)
Trainable params: 108311810 (413.18 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Then we call model.fit to perform the actual training.  Note that many times, because of what the model has learned about language in it pre-training phase we can limit our training to a small number of epochs, sometimes as few as one or two.

In [ ]:
model.fit(train_tf_dataset, validation_data=eval_tf_dataset, epochs=2)

Epoch 1/2
125/125 [==============================] - 208s 1s/step - loss: 0.5742 - sparse_categorical_accuracy: 0.6680 - val_loss: 0.3874 - val_sparse_categorical_accuracy: 0.8500
Epoch 2/2
125/125 [==============================] - 157s 1s/step - loss: 0.3198 - sparse_categorical_accuracy: 0.8640 - val_loss: 0.4010 - val_sparse_categorical_accuracy: 0.8240


We'll be using TensorFlow and BERT in some class assignments but instead of using AutoClasses we'll actually add our own layers on top of BERT so we can build intuition about how it works.